# Model Report

In [ ]:
import sys, os
import pandas as pd
import seaborn as sns

# get access to the src directory
sys.path.append((os.path.abspath("workflow")))

## Read in data

In [ ]:
import pickle
from gzip import GzipFile

models = {}
for fn in snakemake.input:
	with GzipFile(fn, "rb") as f:
		m = pickle.load(f)
	models[m.id_] = m

## ROC

In [ ]:
from sklearn.metrics import roc_curve, auc

# Wrangle
plot_df = []
for id, m in models.items():
	for fold in m.folds_:
		roc = pd.DataFrame()
		roc["fpr"], roc["tpr"], roc["threshold"] = roc_curve(m.folds_[fold]["test"]["label"], m.folds_[fold]["test"]["proba_KNRGL"], pos_label="KNRGL")
		roc["fold"] = fold; roc["model"] = id; roc["auc"] = auc(roc["fpr"], roc["tpr"])
		plot_df.append(roc)

plot_df = pd.concat(plot_df).groupby(["model","threshold"]).mean(["fpr","fpr","auc"]).reset_index()

# Plot 
sns.set_style("ticks")
fig = sns.relplot(
    data=plot_df,
    x="fpr",
    y="tpr",
    hue="model",
    kind="line",
)
fig.set(xlim=(0, 1), ylim=(0, 1))
sns.despine()

## Theoretical PPV

In [ ]:
# from sklearn.metrics import confusion_matrix
# bal = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

# # Wrangle
# plot_df = []
# for id, m in models.items():
# 	for fold in m.folds_:
# 		for b in bal:
# 			cm = confusion_matrix(m.folds_[fold]["test"]["label"], m.folds_[fold]["test"]["pred"], normalize="true")

## Feature Importances

In [ ]:
from functools import reduce

def get_fi(estimator):
	df = pd.DataFrame()
	df["feature"] = estimator.feature_names_in_
	df["importance"] = estimator.feature_importances_
	return df

fi_list = []
for id, m in models.items():
	if "feature_importances_" not in dir(m.folds_[0]["clf"]._final_estimator):
		print(f"{id} does not have feature importances")
		continue
	fi = [get_fi(m.folds_[fold]["clf"]._final_estimator) for fold in m.folds_]
	df = reduce(lambda x, y: pd.merge(x, y, on="feature"), fi).set_index("feature")
	df["mean_importance"] = df.mean(axis=1)
	df["model_id"] = id
	df.sort_values("mean_importance", ascending=False, inplace=True)
	df.reset_index(inplace=True)
	fi_list.append(df)

fi_df = pd.concat(fi_list)
fig = sns.FacetGrid(
	fi_df, col="model_id", sharey=False, sharex=False, height=12, aspect=0.6
)
fig.map_dataframe(
	sns.pointplot,
	x="mean_importance",
	y="feature",
	join=False,
)
sns.despine()